<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->




# **P1 - RA ÍMPAR**

In [ ]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'Vinicius Urias da Cruz, 20006012' #@param {type:"string"}






In [3]:
import pandas as pd
import numpy as np
import random
path = 'https://github.com/Rogerio-mack/work/raw/main/data/'

# **Case**: Airbnb

Neste case, analise dados sobre imóveis alugados pela plataforma Airbnb em NYC no ano de 2019.


In [60]:
df = pd.read_csv(path + 'AB_NYC_2019.csv')
df = df.drop_duplicates()
df.head()
# df.dtypes

id                                            name  host_id  \
0   20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1   25235               Large 2 Bedroom Great for Groups!    87773   
2   76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3  135465             Garden apartment close to Manhattan   663879   
4  169002                Modern Space in Charming Pre-war   805344   

     host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0   Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1       Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2       Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3  Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4         Alec           Manhattan              Harlem  40.82411  -73.94934   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   
2  Entire home/apt    169               2                398  2019-06-28   
3  Entire home/apt    170               3                 86  2019-06-20   
4     Private room     65               2                 41  2019-06-16   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               1.57                               1                 0  
1               1.46                               2               137  
2               3.97                               3               182  
3               0.91                               2               286  
4               0.43                               2                59

# Q1. Proprietários com mais imóveis na plataforma

Obtenha os 3 proprietários ("hosts") com mais imóveis sendo anunciados na plataforma, e a respectiva quantidade de imóveis anunciados.


In [12]:
propietario = df['host_name'].value_counts()
propietario.head(3)

host_name
Mark            7
Sonder (NYC)    7
Andrew          6
Name: count, dtype: int64

# Q2B. Preço nos bairros com mais imóveis

 Qual o preço médio dos imóveis, considerando somente os preços dos imóveis cujos bairros (`neighbourhood`) tenham mais de 20 imóveis sendo anunciados?  


In [96]:
# neighbourhood
bairros = df['neighbourhood'].value_counts()
top_bairros = bairros[bairros > 20].index
df_top_bairros = df[df['neighbourhood'].isin(top_bairros)]
preco_medio = df_top_bairros['price'].mean()
preco_medio


np.float64(151.41888619854723)

In [97]:
...

print(f'O preço médio dos imóveis em bairros com mais de 20 anúncios é: USD {preco_medio:.2f}')

O preço médio dos imóveis em bairros com mais de 20 anúncios é: USD 151.42


# Q3B.  Brooklyn *outliers*

Qual o percentual de imóveis de Brooklyn com preços estatisticamente discrepantes (*outliers*) com relação aos preços de todos os imóveis?



In [98]:
df_brooklyn = df[df['neighbourhood_group'].isin(['Brooklyn'])]
Q1 = df_brooklyn['price'].quantile(0.25)
Q3 = df_brooklyn['price'].quantile(0.75)
IQR = Q3 - Q1
inf = (Q1 - 1.5*IQR)
sup = (Q3 + 1.5*IQR)
print(inf,sup) #Não há preço negativo, outlier para baixo inexistente
quantidade_outliers = len(df[df.price > sup])
perc_Brooklyn = (quantidade_outliers/len(df_brooklyn))*100


-65.0 279.0


In [99]:
...

print( f'Percentuais de outliers de preços, Brooklyn: {perc_Brooklyn:.2f} %')


Percentuais de outliers de preços, Brooklyn: 23.33 %


# Q4B. Scores

Nesta tarefa você deve atribuir a cada imóvel um valor de *score* (`df['score']`) baseado nas seguintes regras:

1. Imóveis com `availability_365` > 90 recebem +10 pontos enquanto imóveis com `availability_365` < 5 recebem -5 pontos;
2. Imóveis com o número de reviews >= 10 recebem +5 pontos para cada 10 reviews (por ex. se número de reviews = 20, são atribuídos +10 pontos). Se número de reviews = 0 o imóvel recebe -5 pontos;
3. Imóveis com preço abaixo da média de seu `neighbourhood`	recebem +5 pontos.

Implemente empregando `apply()` ou um laço de programa, como preferir.

In [52]:
preco_medio_bairro = df.groupby('neighbourhood_group')['price'].mean()
def score(row):
    score = 0
    if row['availability_365'] > 90:
        score += 10
    elif row['availability_365'] < 5:
        score -= 5
    if row['number_of_reviews'] >= 10:
        score += 5 * (row['number_of_reviews'] // 10)
    elif row['number_of_reviews'] == 0:
        score -= 5
    avg_price = preco_medio_bairro[row['neighbourhood_group']]
    if row['price'] < avg_price:
        score += 5
    return score

df['score'] = df.apply(score, axis=1)

In [53]:
...

print(f'A soma dos valores de score é: ', df['score'].sum())


A soma dos valores de score é:  11215


# Q5. Enriquecendo os dados

Enriqueça os dados dos imóveis com os valores dos impostos e multas municipais. Cacule a média de impostos e multas de cada grupo de vizinhaça, `neighbourhood_group`. Note que existem imóveis sem impostos e sem multas, esses são imóveis isentos, mas devem ser considerados nos cálculos de média (por exemplo, você pode considerar esses imóveis com valor 0).

In [92]:
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')
df_novo = pd.merge(df, df_taxes, on='id', how='left')
df_novo['value'] = df_novo['value'].fillna(0)
df_impostos = df_novo[df_novo['type'].isin(['taxes'])]
df_multas = df_novo[df_novo['type'].isin(['fines'])]
df_media_impostos = df_impostos.groupby('neighbourhood_group')['value'].mean()
df_media_multas = df_multas.groupby('neighbourhood_group')['value'].mean()
print("Medias de Impostos: ")
print(df_media_impostos.astype(int))
print('\n')
print("Medias de Multas: ")
print(df_media_multas.astype(int))

Medias de Impostos: 
neighbourhood_group
Bronx             851
Brooklyn          858
Manhattan         888
Queens            804
Staten Island    1157
Name: value, dtype: int64


Medias de Multas: 
neighbourhood_group
Bronx            126
Brooklyn         314
Manhattan        314
Queens           322
Staten Island    581
Name: value, dtype: int64


In [94]:
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')
df_taxes.head()

id   type   value
0  35907127  taxes  1315.0
1    804815  taxes     NaN
2  22328585  taxes   900.0
3  27876600  taxes   869.0
4  34976037  taxes  1629.0

# Q6. Quebras

Construa no `Pandas` um "relatório" (`df_report`) como abaixo. Os valores são ordenados `neighbourhood_group`,`neighbourhood`. Cada "página" contém apenas 30 linhas. A cada "quebra" de `neighbourhood_group` é gerada uma linha com preço médio do `neighbourhood_group` e, então, é iniciada uma nova página.

O item `line_id` é opcional mas deve auxiliá-lo nessa tarefa.

Ao final produza a soma dos preços das páginas abaixo...

In [ ]:
sum_price_pages = df_report[ df_report.page_id.isin([1,24,25,26,28]) ].groupby('page_id').price.sum().astype(int).sum()

print(f'A soma dos preços das páginas 1, 24, 25 e 28 é: {sum_price_pages}')

#### Aqui um *sample* do report.

In [ ]:
df_report.iloc[-10::,:]

neighbourhood_group  neighbourhood       price  page_id  line_id
795              Queens       Woodside   75.000000       28       17
796              Queens       Woodside  120.000000       28       18
797              Queens           MEAN   92.227848       28       19
798       Staten Island  Randall Manor   99.000000       29        1
799       Staten Island    South Beach   20.000000       29        2
800       Staten Island    South Beach  100.000000       29        3
801       Staten Island     St. George  130.000000       29        4
802       Staten Island      Todt Hill  135.000000       29        5
803       Staten Island  Tompkinsville   59.000000       29        6
804       Staten Island           MEAN   89.428571       29        7